In [ ]:
!git clone https://github.com/MichalBaryla/medium_csv.git
!cp ./medium_csv/medium.csv ./


In [ ]:
%pip install -q llama-index-finetuning
%pip install -q llama-index-readers-file
%pip install -q llama-index-llms-huggingface
%pip install -q llama-index-embeddings-huggingface
%pip install -q llama-index-vector-stores-faiss
%pip install -q chromadb
%pip install -q llama_index_vector_stores.chroma

In [ ]:
!mkdir ./splitted

In [ ]:
import pandas as pd

f = pd.read_csv("./medium.csv")

pth = './splitted/'
for n,i in enumerate(f.values):
    ff = open(pth+str(n)+'.txt','w')
    ff.write(i[0]+'\n')
    ff.write(i[1])

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from dotenv import load_dotenv
import numpy as np
import os

load_dotenv()

documents = SimpleDirectoryReader("./splitted").load_data()

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=200, chunk_overlap=10)
nodes = text_splitter.get_nodes_from_documents(documents=documents)


In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.core import StorageContext

chroma_client = chromadb.PersistentClient('./medium_chroma1')
chroma_collection = chroma_client.create_collection("test1")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [ ]:
index = VectorStoreIndex(nodes=nodes, storage_context=storage_context, embed_model=embed_model)

In [ ]:
retriever = index.as_retriever(similarity_top_k=5)

In [ ]:
# setup prompts - specific to StableLM
from llama_index.core import PromptTemplate

# This will wrap the default prompts that are internal to llama-index
# taken from https://huggingface.co/Writer/camel-5b-hf
query_wrapper_prompt = PromptTemplate(
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{query_str}\n\n### Response:"
)

In [ ]:
import torch
from llama_index.llms.huggingface import HuggingFaceLLM

llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device_map="cuda",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.bfloat16}
)



In [ ]:
query_engine = index.as_query_engine(llm=llm)
# llm.model

In [ ]:
from llama_index.core import PromptTemplate

template = """Article: {question}

    Answer: You are an AI expert who prepares questions for a test about AI. Questions:"""

new_summary_tmpl = PromptTemplate(template)



new_summary_tmpl_str = (
    "You always say 'Hello my friend' at the beginning of your answer. Below you find data from a database\n"
    "{context_str}\n"
    "Take that context and try to answer the question with it."
    "Query: {query_str}\n"
    "Answer: "
)
new_summary_tmpl = PromptTemplate(new_summary_tmpl_str)


In [ ]:
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": new_summary_tmpl}
)

In [ ]:
prompts_dict = query_engine.get_prompts()
print(prompts_dict)

In [ ]:
question = open(pth+str(0)+'.txt').read()
qs = question[:question.find('**')].replace('\n\n',' ').split('\n')
qs

In [ ]:
ans_llm = query_engine.query("What are some real-world examples of how plotnine can be used to visualize data?")

In [ ]:
ans_llm_r = ans_llm.response.split()
for i in range(0,len(ans_llm_r),10):
  s=''
  for j in ans_llm_r[i:i+10]:
    s+=j+' '

  print(s)
